#### Setup
We first set the notebook to display the output from each code block, <br>
then import the required packages and set the file directory

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from shapely.geometry import Point
import matplotlib.pyplot as plt
from rasterio.mask import mask
from rasterio.plot import show
from rasterio.plot import plotting_extent
from natsort import natsorted
import earthpy.spatial as es
import pandas as pd
import geopandas
import rasterio
import pycrs
import os

file_dir=r'C:/Users/Mark.Rademaker/PycharmProjects/InternshipNaturalis/trait-geo-diverse-dl/pilot'

#### Read in environmental data
Next the environmental variable rasters are read in and their locations are saved in a list 
for stacking.

In [ ]:
#ENVIREM dataset
#create list of raster file locations and a list containing only the names of each raster
list_envirem_files=[]
names_envirem=[]

#add files to these lists
for root, dirs, files in os.walk(file_dir+"/data/GIS/5_deg"):
    for file in files:
        if file.endswith('.tif'):
            list_envirem_files.append(file_dir+"/data/GIS/5_deg/"+file)
            name=file.replace(".tif","")
            names_envirem.append(name)
            names_envirem=natsorted(names_envirem,key=lambda y: y.lower())
            
    
#BIOCLIM dataset
#create list of raster file locations and a list containing only the names of each raster
list_bioclim_files=[]
names_bioclim=[]

for root, dirs, files in os.walk(file_dir+"/data/GIS/wc5"):
    for file in files:
        if file.endswith('.tif'):
            list_bioclim_files.append(file_dir+"/data/GIS/wc5/"+file)
            name=file.replace(".tif","")
            names_bioclim.append(name)
            names_bioclim=natsorted(names_bioclim,key=lambda y: y.lower())


In [ ]:
#Inspect the raster maps
for name in names_envirem:
    raster=rasterio.open(file_dir+"/data/GIS/5_deg/"+name+".tif")
    print(raster.meta) #inspect meta information
    show(raster,title="ENVIREM %s"%name)
    
for name in names_bioclim:
    raster=rasterio.open(file_dir+"/data/GIS/wc5/"+name+".tif") 
    print(raster.meta) #inspect meta information
    show(raster, title="Bioclim %s"%name)

#### Stack the raster files
The raster files can now be stacked to create a single raster file. The names of the variables are stored in a separate text file. <br> 
The output files (as well as the clipped outputs later) will be too large to store on github. Therefore they are stored in a different path.

In [ ]:
#ENVIREM+BIOCLIM dataset
list_envirem_bioclim_files=[]
for item in list_envirem_files:
    list_envirem_bioclim_files.append(item)
for item in list_bioclim_files:
    list_envirem_bioclim_files.append(item)

names_envirem_bioclim=[]
for item in names_envirem:
    names_envirem_bioclim.append(item)
for item in names_bioclim:
    names_envirem_bioclim.append(item)
    
#save the names of the variables in order to list
with open(file_dir+'/data/GIS/env_stacked/variable_list.txt', 'w') as f:
    for item in names_envirem_bioclim:
        f.write("%s\n" % item)      
        
es.stack(list_envirem_bioclim_files, file_dir+'/data/GIS/env_stacked/ENVIREM_BIOCLIM_stacked.tif')
#this stacked file is too large to put on github, so put them in gitignore